In [12]:
import pandas as pd
import numpy as np
import glob
import re
import os
import sys

f=open(r"demo.out.ss8", "r")   
for line in f.readlines():
    if '>' in line:
        pdbname=line.strip().split('>')[1]    
    else:
        num = -1
        i= -1
        sse =''
        for s in line:
            num +=1 
#             print(num)
            sse= sse+s
        with open(r'./SSE/proteindssp/%s_protein.dssp'%(pdbname), "r") as fg:
            for _ in range(28):
                next(fg)
            for line1 in fg:
                if line1[13]=='!':
                    continue
                else:
                    i +=1
                    if i <= num:
                        line1=line1[:16] + sse[i] + line1[17:]
                        with open(r'./SSE/predprotein/%s_protein.dssp'%(pdbname),"a+") as fw:
                            fw.write(line1)
f.close()      

In [15]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import glob
import re
import os

f_g = glob.glob('./SSE/predprotein/*')
f_l = glob.glob('./SSE/predpocket/*')

columns_g = ['residue','chain','aa','structure']
columns_l = ['residue','chain','aa','structure']
structure = ('B', 'C', 'E', 'G', 'H', 'I', 'S', 'T')
aa = ('G', 'A', 'V', 'L', 'I', 'M', 'F', 'P', 'W', 'S', 'T', 'Y', 'C', 'Q', 'N', 'D', 'E', 'K', 'R', 'H', 'X')

c1 = {
    'non_polar' : ('G','A','V','L','I','M','F','P','W'),
    'polar' : ('S','T','Y','C','Q','N'),
    'acidic' : ('D','E'),
    'basic' : ('K','R','H'),
}

def f1(aa, key):
    if aa=='X':
        return 1/len(c1)
    return 1 if aa in c1[key] else 0


c2 = {
    1 : ('A','G','V'),
    2 : ('I','L','F','P'),
    3 : ('Y','M','T','S'),
    4 : ('H','N','Q','W'),
    5 : ('R','K'),
    6 : ('D','E'),
    7 : ('C',)
}

def f2(aa, key):
    if aa=='X':
        return 1/len(c2)
    return 1 if aa in c2[key] else 0


def idx_df_l_init(df_g):
    chains = df_g.chain.unique()
    
    if len(chains) == 1:
        return lambda row: chains[0]+row.residue
    
    i = -1
    cidx = 0
    
    def idx_df_l(row):
        nonlocal i
        nonlocal df_g

        while True:
            i+=1
            row_g = df_g.iloc[i]
            
            if row_g.residue==row.residue:
                if row.aa != row_g.aa:
                    continue
                if row.chain and row.chain!=row_g.chain:
                    continue
                break
            
        return row_g.chain+row.residue
    return idx_df_l

In [19]:
demo = set(pd.read_csv('demo_seq_.csv').id)
dataset = {
    'demo': demo.copy(),
}
columns_s = ['id','seq']
seq_dfs = {
    'demo': pd.DataFrame(columns=columns_s),
}
pocket_dfs = {
    'demo': pd.DataFrame(columns=columns_s),
}
other_seq = set()

for fi_g in tqdm(f_g):
    seq_name = os.path.basename(fi_g)[:4]
    
    flag = True
    phase = None
    for k in dataset:
        if seq_name in dataset[k]:
            dataset[k].remove(seq_name)
            phase = k
            flag = False
            break
    if flag:
        other_seq.add(seq_name)
        continue
    fi_l = os.path.join(os.path.dirname(fi_g),os.path.basename(fi_g)).replace('protein','pocket')
    if not os.path.exists(fi_l):
        f_err.write(f'{fi_l} not exists')
        continue
    
    df_g = pd.DataFrame(columns=columns_g)
    with open(fi_g) as f:
        for line in f:
            if line[13]=='!':
                continue
            else:
                row = pd.Series([line[5:11].strip(),line[11:12].strip(),line[12:14].strip(),line[16:17].strip()],
                           index=columns_g)
            df_g = df_g.append(row, ignore_index=True)
     
    df_g.aa = df_g.aa.apply(lambda x: 'C' if re.match('[A-Z]',x) is None else x)
    df_g.structure = df_g.structure.apply(lambda x: 'C' if re.match('[A-Z]',x) is None else x)

    assert len(df_g.structure.unique())<= 8
    assert len(df_g.aa.unique())<= 21
    
    for key in c1:
        new_feature = df_g.aa.apply(lambda x:f1(x,key))
        df_g = eval(f'df_g.assign({key}=new_feature)')
    
    for key in c2:
        new_feature = df_g.aa.apply(lambda x:f2(x,key))
        df_g = eval(f'df_g.assign(c2_{key}=new_feature)')
    
    for si in structure:
        new_feature = df_g.structure.apply(lambda x:1 if x==si else 0)
        df_g = eval(f'df_g.assign(s2_{si}=new_feature)')
    
    for ai in aa:
        new_feature = df_g.aa.apply(lambda x:1 if x==ai else 0)
        df_g = eval(f'df_g.assign(a_{ai}=new_feature)')
    
    seq_dfs[phase] = seq_dfs[phase].append(pd.Series([seq_name,''.join(df_g.aa)],index=columns_s),ignore_index=True)
    
    df_g = df_g.assign(idx=df_g.chain+df_g.residue)
    
    
    df_l = pd.DataFrame(columns=columns_l)
    with open(fi_l) as f:

        for line in f:
            if line[13]=='!':
                continue
            else:
                row = pd.Series([line[5:11].strip(),line[11:12].strip(),line[12:14].strip(),line[16:17].strip()],
                           index=columns_l)
            df_l = df_l.append(row, ignore_index=True)
    
    df_l.aa = df_l.aa.apply(lambda x: 'C' if re.match('[A-Z]',x) is None else x)
    
    pocket_dfs[phase] = pocket_dfs[phase].append(pd.Series([seq_name,''.join(df_l.aa)],index=columns_s),ignore_index=True)
    
    try:
        df_l = df_l.assign(idx=df_l.apply(idx_df_l_init(df_g),axis=1))
    except:
        print(f"{seq_name} error: {sys.exc_info()[0]}")
        continue
    
    df_g.drop('aa',axis=1,inplace=True)
    df_g.drop('structure',axis=1,inplace=True)
    df_g.drop('residue',axis=1,inplace=True)
    df_g.drop('chain',axis=1,inplace=True)
    
    df_g.to_csv(f'{phase}/global/{seq_name}.csv')
    
    df_l.drop('aa',axis=1,inplace=True)
    df_l.drop('structure',axis=1,inplace=True)
    df_l.drop('residue',axis=1,inplace=True)
    df_l.drop('chain',axis=1,inplace=True)
    
    df_l = df_l.merge(df_g)
    
    df_l.to_csv(f'{phase}/pocket/{seq_name}.csv')
    
    del df_g
    del df_l

for k in pocket_dfs:
    pocket_dfs[k].to_csv(f'{k}_pocket_.csv')


<ipython-input-19-0309838b1f93>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fi_g in tqdm(f_g):


  0%|          | 0/1 [00:00<?, ?it/s]